In [1]:
import requests
import re
from bs4 import BeautifulSoup
import pandas as pd
import time
import numpy as np
from datetime import datetime

In [2]:
def html_stripper(text):
    return re.sub('<[^<]+?>', '', str(text))

Тут можно задать параметры поиска.

In [3]:
maxprice = 75000
one = 0
two = 0
three = 1
four = 1
five = 1
mult = 1

In [4]:
links = []

### Парсинг ссылок (еcли нужна подкачка списка ссылок из файла, то можно использовать код ниже).

Здесь мы пройдемся по всем районам (в каждом округе их порядка 10-15). Итого получилось прохождение по 130 поисковым запросам. По каждому запросу получаем выдачу, так как запросы подробные, то выдачи обычно 1-2 страницы, но я на всякий случай прохожусь по первым 5 (page in range(1, 6), можно заменить верхнее значение при желании). Это сказывается следующим образом: когда мы обращаемся по адресу с номером страницы, которая вне диапазона, то циан нас редиректит на первую страницу выдачи. То есть при таких параметрах мы жертвуем временем и много раз считываем одно и то же в страхе пропустить что-то важное. 
С каждой страницы мы собираем все ссылки. За повторения можно не беспокоиться - они будут в любом случае, так как люди могут указать несколько районов для своей квартиры, например, но ниже готовый пулл всех ссылок очищается от дубликатов.

In [5]:
districts = {1: 'NW', 4: 'C', 5:'N', 6:'NE', 7:'E', 8:'SE', 9:'S', 10:'SW', 11:'W'}
zone_bounds = {1: (125, 133), 4: (13, 23), 5: (23, 39), 6:(39, 56), 7:(56, 72), 8:(72, 84), 9:(84, 100), 10:(100, 112), 11:(112, 125)}

for i in districts.keys():
    left = zone_bounds[i][0]
    right = zone_bounds[i][1]
    
    for j in range(left, right):
        zone = 'https://www.cian.ru/cat.php?currency=2&deal_type=rent&district%5B0%5D=' + str(j) + '&engine_version=2&maxprice=' + str(maxprice) + '&offer_type=flat&p={}&room1=' + str(one) + '&room2=' + str(two) + '&room3=' + str(three) + '&room4=' + str(four) + '&room5=' + str(five) + '&room6='+ str(mult) + '&type=4'
        for page in range(1, 6):
            broken = False
            if broken:
                break
            while True:
                try:
                    page_url =  zone.format(page)

                    search_page = requests.get(page_url)
                    search_page = search_page.content
                    search_page = BeautifulSoup(search_page, 'lxml')

                    flat_urls = search_page.findAll('div', attrs = {'ng-class':"{'serp-item_removed': offer.remove.state, 'serp-item_popup-opened': isPopupOpen}"})
                    flat_urls = re.split('https://www.cian.ru/rent/flat/|/" ng-class="', str(flat_urls))


                    for link in flat_urls:
                        if link.isdigit():
                            #print(link)
                            links.append(link)
                    break
                except KeyboardInterrupt:
                    print('^C')
                    broken = True
                    break
                except:
                    print('bad connection')
                    time.sleep(1)
                    continue                    

In [6]:
len(links)

12122

In [7]:
links = list(set(links))
links = [int(i) for i in links]

In [8]:
len(links)

2518

Ну и сделаем бэкап ссылок

In [9]:
backup = links[:]
backup = pd.DataFrame(backup)

date = str(datetime.today())
for i in date:
    if date[i] == ' ':
        date[i] = '_'

backup.to_csv('backup_links_cian_rent_%s.csv' %date, index = False)

### Это альтернативный вариант: подкачка списка ссылок из файла.

In [10]:
#links = pd.read_csv('/home/tatiana/Study/backup_links_cian_rent_2017-01-19_02:02:42.283942.csv')
#links = links['0']

### Функции для сбора данных.

Будем собирать: цену, gps-координаты (дополнительно оставляю подсчет расстояния до центра города в километрах, но не включаю его в сам парсер, включить можно добавлением строк distance = getDistance(flat_page) info.append(distance) после всех остальных вызовов в тело функции getInfo(), а также строку 10: 'Distance' в переименование колонок), число комнат, общую площадь (оставлю также дополнительно площадь кухни, добавлять соответственно), площадь комнат + изолированность, метро, адрес.

In [11]:
def getPrice(flat_page):
    price = flat_page.find('div', attrs={'class':'object_descr_price'})
    price = re.split('<div>|руб|\W', str(price))
    price = "".join([i for i in price if i.isdigit()][-4:])

    dollar = '808080'
    if dollar in price:
        price = price[6:]
        
    return int(price)

In [12]:
def getCoords(flat_page):
    coords = flat_page.find('div', attrs={'class':'map_info_button_extend'}).contents[1]
    coords = re.split('&amp|center=|%2C', str(coords))
    coords_list = []
    for item in coords:
        if item[0].isdigit():
            coords_list.append(item)
    lat = float(coords_list[0])
    lon = float(coords_list[1])
    return lat, lon

In [13]:
from math import radians, cos, sin, asin, sqrt
AVG_EARTH_RADIUS = 6371

def haversine(point1, point2):

    # извлекаем долготу и широту
    lat1, lng1 = point1
    lat2, lng2 = point2

    # переводим все эти значения в радианы
    lat1, lng1, lat2, lng2 = map(radians, (lat1, lng1, lat2, lng2))

    # вычисляем расстояние по формуле
    lat = lat2 - lat1
    lng = lng2 - lng1
    d = sin(lat * 0.5) ** 2 + cos(lat1) * cos(lat2) * sin(lng * 0.5) ** 2
    h = 2 * AVG_EARTH_RADIUS * asin(sqrt(d))

    return h 

In [14]:
def getDistance(coords):
    MSC_POINT_ZERO = (55.755831, 37.617673)
    return haversine(MSC_POINT_ZERO, coords)

In [15]:
def getRoom(flat_page):
    rooms_n = flat_page.find('div', attrs={'class':'object_descr_title'})
    rooms_n = html_stripper(rooms_n)
    room_number = ''
    flag = 0
    for i in re.split('-|\n', rooms_n):
        if 'много' in i:
            flag = 1
            break
        elif 'комн' in i:
            break
        else:
            room_number += i
    
    if (flag):
        room_number = '6'
    room_number = "".join(room_number.split())
    return int(room_number)

In [16]:
def getFloor(flat_page):
    table = flat_page.find('table', attrs = {'class':'object_descr_props'})
    table = html_stripper(table)
    
    floor_is = 0
    
    building_block = re.split('Этаж|Тип продажи', table)[1]
    floor_block = re.split('\xa0/\xa0|\n|\xa0', building_block)
    
    for i in range(1, len(floor_block[2]) + 1):
        if(floor_block[2][-i].isdigit()):
            floor_is += int(floor_block[2][-i]) * 10**(i - 1)
            
    return floor_is

In [44]:
def myStrToFloat(string):
    delimiter = 0
    value = 0
    for i in range(0, len(string)):
        if string[i] == ',':
            delimiter = i
    if delimiter == 0:
        return int(string)
    for i in range(0, delimiter):
        value += int(string[delimiter - i - 1]) * 10 ** i

    for i in range(1, len(string) - delimiter):
        value += (int(string[delimiter + i]) * (10 ** (-i)))
    return value

In [18]:
def getTotsp(flat_page):
    table = flat_page.find('table', attrs = {'class':'object_descr_props'})
    table = html_stripper(table)
    
    space_block = re.split('Общая площадь', table)[1]
   
    total = re.split('Площадь комнат', space_block)[0]
    total_space = re.split('\n|\xa0', total)[3]
    if total_space.isdigit():
        total_space = int(total_space)
    else:
        total_space = myStrToFloat(total_space)
            
    return total_space

In [19]:
def getKitsp(flat_page):
    table = flat_page.find('table', attrs = {'class':'object_descr_props'})
    table = html_stripper(table)
    
    space_block = re.split('Общая площадь', table)[1]
    optional_block = re.split('Жилая площадь', space_block)[1]
    
    kitchen_space = np.nan
    
    if 'Площадь кухни' in optional_block:
        kitchen_block = re.split('Площадь кухни', optional_block)[1]
        if re.split('\n|\xa0', kitchen_block)[2] != '–':
            if re.split('\n|\xa0', kitchen_block)[2].isdigit():
                kitchen_space = int(re.split('\n|\xa0', kitchen_block)[2])
            else:
                kitchen_space = myStrToFloat(re.split('\n|\xa0', kitchen_block)[2])
            
    return kitchen_space

In [47]:
def getRoomsSp(flat_page):
    isolated = True
    
    table = flat_page.find('table', attrs = {'class':'object_descr_props'})

    table = re.sub('<[^<]?>', '', str(table))

    space_block = re.split('Общая площадь', table)[1]
    
    rooms = re.split('Площадь комнат:', space_block)[1]
    rooms = re.split('Жилая площадь', rooms)[0]
    rooms = re.split('-->', rooms)[1][0]

    if(rooms == '–'):
        return np.nan, np.nan  
    
    rooms_space = re.split('all_rooms_area', table)[1]
    rooms_space = re.split('\xa0', rooms_space)[0]
    rooms_space = rooms_space[3:]
    
    space = ''
    spaces = []
    finished = True
    
    for i in range(len(rooms_space)):
        
        if rooms_space[i] == ' ':
            continue

        elif (not(rooms_space[i].isdigit()) and not(finished)):
            if rooms_space[i] == ',':
                space += rooms_space[i]
            else:
                #print(space)
                spaces.append(myStrToFloat(space))
                space = ''
                finished = True

        elif (rooms_space[i].isdigit()):
            finished = False
            space += rooms_space[i]
            
        elif (not(rooms_space[i].isdigit()) and finished):
            continue
    
    if not finished:
        spaces.append(myStrToFloat(space))
    
    if '+' in rooms_space:
        isolated = False
        
    return spaces, isolated

In [21]:
def getMetro(flat_page):
    
    if (flat_page.find('div', attrs={'class':'object_item_metro_name'}) == None):
        metro_station = flat_page.find('div', attrs={'class':'object_descr_metro'})
        metro_station = re.split('_blank">|,</a>', str(metro_station))
        stations = []
        for i in range(len(metro_station)):
            if (i % 2 != 0):
                stations.append(metro_station[i])
    else:
        metro_station = flat_page.find('div', attrs={'class':'object_item_metro_name'})
        stations = re.split('_blank">|,</a>', str(metro_station))[1]
    
    return stations

In [22]:
def getAddress(flat_page):
    address = flat_page.find('h1', attrs={'class':'object_descr_addr'})
    address = re.split('<a href=', str(address))

    if 'район' in address[2]:
        if len(address) > 4:
            street = address[3]
            house = address[4]
        else:
            street = address[3]
            house = ''
    else:
        street = address[2]
        house = address[3]
    
    
    if len(address) > 4:
        street = re.split('_blank">|</a>', str(street))[1]
        house = re.split('_blank">|</a>', str(house))[1]
    else:
        street = re.split('_blank">|</a>', str(street))[1]
    
    return street + ' ' + house

Эти выведены в отдельные исключительно для удобства при чтении бОльших, содержащих эти части кода, функций

In [23]:
def getFlatPage(link):
    
    flat_url = 'http://www.cian.ru/rent/flat/' + str(link) + '/'
    flat_page = requests.get(flat_url)
    flat_page = flat_page.content
    flat_page = BeautifulSoup(flat_page, 'lxml')
    
    return flat_page

In [24]:
def getFlatUrl(page):
    page_url =  district.format(page)

    search_page = requests.get(page_url)
    search_page = search_page.content
    search_page = BeautifulSoup(search_page, 'lxml')

    flat_url = search_page.findAll('div', attrs = {'ng-class':"{'serp-item_removed': offer.remove.state, 'serp-item_popup-opened': isPopupOpen}"})
    flat_url = re.split('http://www.cian.ru/rent/flat/|/" ng-class="', str(flat_url))
    
    return flat_url

getInfo() возвращает полную информацию по квартире, вызывая внутри себя вышеперечисленные функции для определения значений признаков

In [25]:
def getInfo(link):
    
    #params = [getPrice, getCoords, getRoom, getFloor, getTotsp, getRoomsSp, getMetro, getAddress]
    flat_page = getFlatPage(link)
    info = []
    
    rooms_space, isolated = getRoomsSp(flat_page)
    info.append(rooms_space)
    info.append(isolated)
    
    price = getPrice(flat_page)
    info.append(price)
    
    coords = getCoords(flat_page)
    info.append(coords)
    
    rooms = getRoom(flat_page)
    info.append(rooms)
    
    floor = getFloor(flat_page)
    info.append(floor)
    
    totsp = getTotsp(flat_page)
    info.append(totsp)
    
    metro = getMetro(flat_page)
    info.append(metro)
    
    address = getAddress(flat_page)
    info.append(address)
    

    info.append(link)
    
    return info

### Фукция парсера.

In [106]:
def cianParser(links, left, right, bunch_size):
    if right <= left:
        print('right border is expected to be greater than the left one. change values pls')
        return 
    if right >= len(links):
        print('right border is out of range')
        return 
    N = 10
    if (right - left) % bunch_size == 0:
        bunch_count = (right - left) // bunch_size
    else:
        bunch_count = (right - left) // bunch_size + 1
    apartments = []
    broken = False
    fails_count = 0
    for i in range(left, right):
        counter = 0
        while True:
            if counter >= N:
                fails_count += 1
                print('getInfo() fail (or bad connection status), flat will be missing in the dataset,',   
                      'position:', i, 'link: https://cian.ru/rent/flat/' + str(links[i]))
                break
            
            try:
                apartments.append(getInfo(links[i]))
                if (i + 1 - left) % bunch_size == 0:
                    if (i + 1 - left) != 0:
                        print((i + 1 - left) // bunch_size, 'bunch of', bunch_count, 'is done')
                break
            except KeyboardInterrupt:
                print('interrupted on', i, 'row',) 
                print('^C')
                broken = True
                print('fails count is', fails_count)
                return pd.DataFrame(apartments)
                break
            except:
                counter += 1
                time.sleep(1)
                continue

    if not ((i + 1 - left) % bunch_size == 0): 
        print(bunch_count, 'bunch of', bunch_count, 'is done')
    print('fails count is', fails_count)    
    return pd.DataFrame(apartments)

## Запуск на полном списке ссылок.

In [107]:
len(links)

2518

Можно изменить размер банча. Он влияет на вывод в процессе парсинга. Прервать процесс можно обычным kernel interrupt, выведется номер строки, на которой остановилась функция. После прерывания готовые данные для квартир до этой строки будут доступны в переменной data.

Могут возникать ошибки. Но парсер на них падать не должен, в случае проблем (плохого соединения или падения getInfo() на конкретной квартире) он попробует N = 10 раз, в случае неудачи выведет сообщение об ошибке и перейдет к следующей ссылке.
Проверить, в чем была проблема для конкретной квартиры, можно кодом ниже.

Чтобы запустить на неполном списке, достаточно указать нужные границы подсета в качестве параметров функции cianParser.

In [108]:
bunch_size = 20

print('started')
data = cianParser(links, 0, len(links), bunch_size)
print('finished')

started
right border is out of range
finished


In [49]:
data.head()

,0,1,2,3,4,5,6,7,8,9
0,"[14, 14, 18]",True,50000,"(55.638861, 37.349174)",3,17,80,[Румянцево],ул. Шолохова 7,152501253
1,"[20, 15, 14]",True,75000,"(55.645893, 37.477022)",3,17,77,"[Тропарево, Юго-Западная]",Ленинский просп. 127,152501046
2,"[18, 14, 12]",True,45000,"(55.871006, 37.562874)",3,11,68,[Петровско-Разумовская],Дубнинская ул. 22К1,152756976
3,"[20, 16, 12]",True,65000,"(55.74226, 37.42465)",3,6,70,[Молодежная],Рублевское ш. 99К4,149493753
4,"[24, 12.5, 15.5]",True,50000,"(55.68296, 37.848529)",3,6,78,[Жулебино],Привольная ул. 65К3,151862388


Проверка квартиры

In [56]:
#необходимо присвоить fail_link интовое значение (обычно девятизначное) из вывода об ошибке
#fail_link = 
#getInfo(fail_link)

In [50]:
data.shape

(27, 10)

In [51]:
data.rename(columns={
                              0: 'RoomsSp', 
                              1: 'Isolated', 
                              2: 'Price', 
                              3: 'GPS', 
                              4: 'RoomsNum', 
                              5: 'Floor', 
                              6: 'TotSp', 
                              7: 'Metro', 
                              8: 'Address', 
                              9: 'Link'
        
                         }, inplace = True)

In [52]:
data.head()

,RoomsSp,Isolated,Price,GPS,RoomsNum,Floor,TotSp,Metro,Address,Link
0,"[14, 14, 18]",True,50000,"(55.638861, 37.349174)",3,17,80,[Румянцево],ул. Шолохова 7,152501253
1,"[20, 15, 14]",True,75000,"(55.645893, 37.477022)",3,17,77,"[Тропарево, Юго-Западная]",Ленинский просп. 127,152501046
2,"[18, 14, 12]",True,45000,"(55.871006, 37.562874)",3,11,68,[Петровско-Разумовская],Дубнинская ул. 22К1,152756976
3,"[20, 16, 12]",True,65000,"(55.74226, 37.42465)",3,6,70,[Молодежная],Рублевское ш. 99К4,149493753
4,"[24, 12.5, 15.5]",True,50000,"(55.68296, 37.848529)",3,6,78,[Жулебино],Привольная ул. 65К3,151862388


In [112]:
date = str(datetime.today())
for i in date:
    if date[i] == ' ':
        date[i] = '_'
data.to_csv('cian_rent_%s.csv' %date, index = False)